In [4]:
from dotenv import load_dotenv
import os
from pathlib import Path

# Direct path specification
env_path = Path('../backend/.env').resolve()  # or Path('../backend/.env') depending on your directory structure

# Load environment variables from .env file with explicit path
load_dotenv(dotenv_path=env_path)

# Access your API key
api_key = os.getenv("EMBER_API_KEY")

# Verify it loaded (don't print the actual key in your notebook!)
print("API key loaded:", "✓" if api_key else "✗")
print(f"Looking for .env file at: {env_path}")

API key loaded: ✓
Looking for .env file at: /Users/allison/workspace/ai playground/steering-interface/backend/.env


In [5]:
import goodfire

In [6]:
client = goodfire.Client(
    api_key=api_key,
)

In [7]:
response =  client.chat.completions.create(
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    model="meta-llama/Llama-3.3-70B-Instruct"
)

print(response.choices[0].message["content"])

The capital of France is Paris.


In [8]:
# Create a variant

vDefault = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")
print(vDefault)

Variant(
   base_model=meta-llama/Llama-3.3-70B-Instruct,
   edits={
   }
   scopes={
   }
)


In [ ]:
# Search features

feature = client.features.search("ELI5", model=vDefault, top_k=10)
print(feature)

FeatureGroup([
   0: "Simplified explanations of complex concepts",
   1: "Explaining complex topics in simpler terms to help understanding",
   2: "Explaining complex concepts using simple, accessible language",
   3: "Breaking down technical concepts into simpler explanations",
   4: "Breaking down complex topics into simpler explanations",
   5: "Breaking down complex topics into simpler explanations",
   6: "The assistant is providing a simplified technical explanation",
   7: "The assistant is simplifying a complex technical concept",
   8: "Making complex concepts understandable to others",
   9: "Detailed explanation of inner workings or processes"
])


In [17]:
inspector = client.features.inspect(
    [
        {"role": "user", "content": "What do you think about pirates and whales"},
        {"role": "assistant", "content": "I think pirates are cool and whales are cool"}
    ],
    model=vDefault
)

# Get top activated features
for activation in inspector.top(k=5):
    print(f"{activation.feature.label}: {activation.activation}")

Technical setup and configuration states in experimental procedures: 7
The assistant should engage with pirate-themed content or roleplay as a pirate: 7
The assistant acknowledges its AI limitations while pivoting to what it can offer: 7
Detailed personification and attribute description in formal writing: 7
Setting up detailed writing or description tasks: 5
